In [134]:
# работа с датафреймами и numpy-массивами
import pandas as pd
import numpy as np

# подготовка данных к обучению
from sklearn.model_selection import train_test_split

# обучение модели
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# оценка точности модели
from sklearn import metrics

# удаление стоп-слов
import nltk

# лемматизация
import pymorphy2

# Отрисовка графиков
import matplotlib.pyplot as plt
import seaborn as sns

# Регулярные выражения
import re

### Функции

Лемматизация

In [135]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    res = list()
    for word in text:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

Обработка текста

In [136]:
def cleanText(string):
    # Приводим буквы к нижнему регистру
    string = string.lower()
    # Убираем ссылки 
    string = re.sub(r"http\S+", "", string)
    # Убираем символы переноса строки
    string = string.replace('\\n', '')
    string = string.replace('\\', '')
    # Заменяем букву "ё" на "е"
    string = str.replace(string,'ё','е')
    # Ищем все буквенные конструкции и представляем их в виде токенов
    prog = re.compile('[А-Яа-яA-Za-z]+')
    words = prog.findall(string)
    # Удаляем стоп-слова
    stopwords = nltk.corpus.stopwords.words('russian')
    words = [w for w in words if w not in stopwords]
    ## Лемматизация и представление массива слов в виде строки
    resultString = ' '.join(lemmatize(words))
    return resultString

Оценка точности на одном наборе данных с помощью сплита

In [137]:
def processed_with_split(text, target):
    # Обработка текста
    text = list(map(cleanText, text))
    # Разбиение выборки на обучающую и тестовую
    X_train, X_test, y_train, y_test = train_test_split(text, target, test_size=.3, stratify=target, shuffle = True, random_state=0)
    # Обучение модели
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()), # Prod feature: tokenizer=cleanText
            ("classifier", LinearSVC())]
        )
    clf_pipeline.fit(X_train, y_train)
    # Предсказание
    predict = clf_pipeline.predict(X_test)
    # Оценка точности 
    f1_base = metrics.f1_score(y_test, predict)
    return f1_base

Оценка точности с помощью двух наборов данных

In [138]:
def processed(train_text, train_target, test_text, test_target):
    # Обработка текста
    train_text = list(map(cleanText, train_text))
    test_text = list(map(cleanText, test_text))
    # Обучение модели
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()), # Prod feature: tokenizer=cleanText
            ("classifier", LinearSVC())]
        )
    clf_pipeline.fit(train_text, train_target)
    # Предсказание
    predict = clf_pipeline.predict(test_text)
    # Оценка точности 
    f1_base = metrics.f1_score(test_target, predict)
    return f1_base

## Рассчет точности

### Загрузка наборов

Набор с Подслушано

In [152]:
podslushano_labeled = pd.read_csv('podslushano_labeled.csv')
podslushano_labeled = podslushano_labeled.drop('Unnamed: 0', axis=1)
podslushano_labeled = podslushano_labeled.fillna(0)
text_podslushano = np.array(podslushano_labeled['Text'].values)
target_podslushano = podslushano_labeled['Target'].astype(int).values
print('Количество деструктивных комментариев:', len(podslushano_labeled[podslushano_labeled['Target'] == 1]))
print('Количество недеструктивных комментариев:', len(podslushano_labeled[podslushano_labeled['Target'] == 0]))

Количество деструктивных комментариев: 1978
Количество недеструктивных комментариев: 11144


Набор с Пикабу

In [153]:
pikabu_labeled = pd.read_csv('labeled.csv')
text_pikabu = np.array(pikabu_labeled['comment'].values)
target_pikabu = pikabu_labeled['toxic'].astype(int).values
print('Количество деструктивных комментариев:', len(pikabu_labeled[pikabu_labeled['toxic'] == 1]))
print('Количество недеструктивных комментариев:', len(pikabu_labeled[pikabu_labeled['toxic'] == 0]))

Количество деструктивных комментариев: 4826
Количество недеструктивных комментариев: 9586


### Оценка точности классификации набора подслушано (модель обучена на наборе подслушано)

In [55]:
print(processed_with_split(text_podslushano, target_podslushano))

0.4587737843551797


### Оценка точности классификации набора пикабу (модель обучена на наборе подслушано)

In [59]:
print(processed(text_podslushano, target_podslushano, text_pikabu, target_pikabu))

0.39788787078738935


### Оценка точности классификации набора пикабу (модель обучена на наборе пикабу)

In [60]:
print(processed_with_split(text_pikabu, target_pikabu))

0.7955307262569832


### Оценка точности классификации набора подслушано (модель обучена на наборе пикубу)

In [61]:
print(processed(text_pikabu, target_pikabu, text_podslushano, target_podslushano))

0.2619867549668874


### Оценка точности на объединенных наборах послушано и пикабу

In [64]:
print(processed_with_split(np.append(text_pikabu, text_podslushano), np.append(target_pikabu, target_podslushano)))

0.6169471851421938


# Чистка набора

In [167]:
def balansing_for_word(df, word):
    i = 0
    bad_string = []
    while i < len(df):
        if df['Text'][i].find(word) != -1:
            bad_string.append(df['Text'][i])
        i += 1
    for i in bad_string:
        index_list_to_del = df.index[df['Text'] == i].tolist()
        df = df.drop(index_list_to_del)
        df = df.reset_index(drop=True)
    return df

In [173]:
podslushano_labeled = pd.read_csv('podslushano_labeled.csv')
podslushano_labeled = podslushano_labeled.drop('Unnamed: 0', axis=1)
podslushano_labeled = podslushano_labeled.fillna(0)
podslushano_labeled = podslushano_labeled.reset_index(drop=True)
len(podslushano_labeled)

13122

In [175]:
text_podslushano = np.array(podslushano_labeled['Text'].values)
target_podslushano = podslushano_labeled['Target'].astype(int).values
print('Количество деструктивных комментариев:', len(podslushano_labeled[podslushano_labeled['Target'] == 1]))
print('Количество недеструктивных комментариев:', len(podslushano_labeled[podslushano_labeled['Target'] == 0]))

Количество деструктивных комментариев: 1978
Количество недеструктивных комментариев: 11131


In [176]:
print(processed_with_split(text_podslushano, target_podslushano))

0.44208289054197664
